In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import BernoulliNB 
from sklearn.metrics import accuracy_score
import pickle

In [35]:
# 1. Muat Dataset
try:
    df = pd.read_csv('Data Prediksi.csv')
except FileNotFoundError:
    print("Error: File 'Data Prediksi.csv' tidak ditemukan.")
    exit()

print("Dataset berhasil dimuat.")

# 2. Pisahkan Fitur (X) dan Target (Y)
X = df.drop('TARGET', axis=1)
Y = df['TARGET']

print("Pemeriksaan data:")
print(f"Jumlah Fitur (Gejala): {X.shape[1]}")
print(f"Kelas Penyakit Unik: {Y.unique()}")

Dataset berhasil dimuat.
Pemeriksaan data:
Jumlah Fitur (Gejala): 20
Kelas Penyakit Unik: ['ALLERGY' 'COLD' 'COVID' 'FLU']


In [37]:
# 3. Encoding Target (Y)
encoder = LabelEncoder()
Y_encoded = encoder.fit_transform(Y)

# 4. Bagi Data (Train/Test Split)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y_encoded, test_size=0.2, stratify=Y_encoded, random_state=2
)

print(f"Data latih: {X_train.shape[0]} baris, Data tes: {X_test.shape[0]} baris")

Data latih: 80000 baris, Data tes: 20000 baris


In [39]:
# 5. Latih Model (Menggunakan Bernoulli Naive Bayes)
classifier = BernoulliNB()  
classifier.fit(X_train, Y_train)

print("Model Naive Bayes berhasil dilatih.")

ValueError: Input X contains NaN.
BernoulliNB does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [9]:
# 6. Evaluasi Model
Y_pred = classifier.predict(X_test)
test_accuracy = accuracy_score(Y_test, Y_pred)
print(f"Akurasi Model pada Data Tes: {test_accuracy * 100:.2f}%")

Akurasi Model pada Data Tes: 96.20%


In [11]:
# 7. Simpan Model dan Metadata untuk Streamlit
model_filename = 'prediksi_penyakit_model.sav'
pickle.dump(classifier, open(model_filename, 'wb'))
print(f"Model disimpan sebagai: {model_filename}")

class_names_filename = 'class_names.sav'
pickle.dump(encoder.classes_, open(class_names_filename, 'wb'))
print(f"Nama kelas disimpan sebagai: {class_names_filename}")

feature_names_filename = 'feature_names.sav'
pickle.dump(list(X.columns), open(feature_names_filename, 'wb'))
print(f"Nama fitur disimpan sebagai: {feature_names_filename}")

print("\n--- Persiapan Deployment Selesai ---")
print("Anda sekarang memiliki 3 file .sav yang siap digunakan oleh Streamlit.")

Model disimpan sebagai: prediksi_penyakit_model.sav
Nama kelas disimpan sebagai: class_names.sav
Nama fitur disimpan sebagai: feature_names.sav

--- Persiapan Deployment Selesai ---
Anda sekarang memiliki 3 file .sav yang siap digunakan oleh Streamlit.


In [19]:
import pickle
import pandas as pd

# 1. MUAT MODEL YANG SUDAH DILATIH DARI FILE
# Pastikan ini dijalankan terlebih dahulu
model = pickle.load(open('prediksi_penyakit_model.sav', 'rb'))

# 2. SEKARANG ANDA BISA MEMBUAT DATA CONTOH
contoh_gejala = {
    'BATUK': [0],
    'NYERI_OTOT': [0],
    'KELELAHAN': [1],
    'SAKIT_TENGGOROKAN': [1],
    'HIDUNG_MELER': [0],
    'HIDUNG_TERSUMBAT': [0],
    'DEMAM': [0],
    'MUAL': [0],
    'MUNTAH': [0],
    'DIARE': [0],
    'SESAK_NAPAS': [0],
    'SULIT_BERNAPAS': [0],
    'KEHILANGAN_INDERA_PERASA': [1],
    'KEHILANGAN_INDERA_PENCIUMAN': [1],
    'MATA_GATAL': [0],
    'HIDUNG_GATAL': [0],
    'MULUT_GATAL': [0],
    'TELINGA_BAGIAN_DALAM_GATAL': [0],
    'BERSIN': [0],
    'MATA_MERAH': [0]
}

df_baru = pd.DataFrame(contoh_gejala)

# 3. BARIS INI SEKARANG AKAN BERHASIL
prediksi_angka = model.predict(df_baru)

# (Opsional) Muat nama kelas untuk hasil yang lebih baik
class_names = pickle.load(open('class_names.sav', 'rb'))
nama_penyakit = class_names[prediksi_angka[0]]

print(f"\nPrediksi penyakit pasien baru: {nama_penyakit}")


Prediksi penyakit pasien baru: COLD
